In [31]:
# Basic set-up
p = 3       # Choose the prime p to calculate with
f = 1       # The degree of the residual field extension
e = p - 1   # The ramification index
q = p^f

# # Define a number field called L by adjoining a root t of the polynomial equation x^e = p:
# L.<Pi> = NumberField(x^e - p)   
# # Thus L = QQ(p^{1/e}). Other number fields can be defined similarly by adjoining roots of other polynomials.
# o_L = L.ring_of_integers()
o_L.<Pi> = Zp(3, prec = 10, type = 'capped-rel', print_mode = 'series').ext(x^2-3)
L.<Pi> = Qp(3, prec = 10, type = 'capped-rel', print_mode = 'series').ext(x^2-3)

# Do linear algebra over the ring of polynomials L[X] in one variable X with coefficients in the field L:
L_X.<X> = L[]
L_Y.<Y> = L[]

In [32]:
# The subroutine Dmatrix calculates the following sparse matrix of coefficients.
# Let D[k,n] be equal to k! times the coefficient of Y^k in the polynomial P_n(Y). 
# I compute this using the useful and easy recursion formula 
#      D[k,n] = \sum_{r \geq 0} \pi^{-r} D[k-1,n-q^r] 
# that can be derived from Laurent’s Prop 1.20 of "outline9".
# The algorithm is as follows: first make a zero matrix with S rows and columns (roughly, S is (q-1)*Size), 
# then quickly populate it one row at a time, using the recursion formula.

def Dmatrix(S):
    D = matrix(L, S,S)
    D[0,0] = 1
    for k in range(1,S):
        for n in range(k,S):
            r = 0
            while n >= q^r:
                D[k,n] = D[k,n] + D[k-1,n-q^r]/Pi^r  # the actual recursion 
                r = r+1
    return D


# \Tau^{(m)} in Definition 10.10 of "bounded21":
def TauMatrix(Size, m):
    D = Dmatrix((q - 1) * (Size + 1))
    R = matrix(L, Size,Size, lambda x,y: D[m + (q-1)*x, m + (q-1)*y])

    # Define a diagonal matrix:
    Diag = matrix(L_X, Size,Size, lambda x,y: kronecker_delta(x,y) * X^x)

    # Compute the inverse of R:
    S = R.inverse()    

    # Compute the matrix Tau using Lemma 10.11 in "bounded21":
    Tau = S * Diag * R
    
    return Tau


def underscore(m, i):
    return m + i*(q-1)


# Compute the matrix ( \sigma_{ij}(Y) ) using Proposition 10.6 of "bounded21":
def sigmaMatrix(Size):
    sigma = matrix(L_Y, Size,Size)
    
    for m in [0 .. q-2]:
        Tau = TauMatrix((q-1)*(Size+1), m)
        j = 0
        while underscore(m, j) < Size:
            i = 0
            while underscore(m, i) < Size:
                sigma[underscore(m, i), underscore(m, j)] = Y^m * Tau[i, j](Y^(q-1))
                i += 1
            j += 1
    
    return sigma

In [33]:
sigmaMatrix(4)

[                                                                                                                      1 + O(Pi^20)                                                                                                                                  0                                                                                                                                  0                                                                                                                                  0]
[                                                                                                                                 0                                                                                                                   (1 + O(Pi^20))*Y                                                                                                                                  0 (2*Pi^-1 + 2*Pi + 2*Pi^3 + 2*Pi^5 + 2*Pi^7 + 2*Pi^9 + 2*Pi^11 + 2*Pi^13 + 2*Pi^1

In [34]:
sigmaMatrix(9)[:, -1]

[                                                                                                                                                                                                                                       0]
[                                                                                                                                                                                                                                       0]
[(2*Pi^-1 + 2*Pi + Pi^3 + 2*Pi^5 + 2*Pi^7 + 2*Pi^9 + 2*Pi^11 + 2*Pi^13 + 2*Pi^15 + O(Pi^17))*Y^8 + (2*Pi^-1 + Pi + Pi^3 + O(Pi^17))*Y^6 + (2*Pi^-1 + Pi + 2*Pi^3 + 2*Pi^5 + 2*Pi^7 + 2*Pi^9 + 2*Pi^11 + 2*Pi^13 + 2*Pi^15 + O(Pi^17))*Y^4]
[                                                                                                                                                                                                                                       0]
[                                                           

In [35]:
# Construct a vwdwo sequence as per Proposition II.2.3 of Cahen-Chabert:
def vwdwo(N):
    u_ = [i for i in range(q)]  # only for L totally ramified
    
    def u(n):
        return o_L(sum(
            u_[n_i] * Pi^i
            for i, n_i in enumerate(Integer(n).digits(base=q))
        ))
    
    return [u(n) for n in [0 .. N]]


# Compute a regular basis (f_n) as per Theorem II.2.7 of Cahen-Chabert:
def regular_basis(N):
    u = vwdwo(N)
    
    def f(n):
        if n == 0:
            return L_Y(1)
        else:
            return prod((Y - u[k]) / (u[n] - u[k]) for k in [0 .. n-1])
    
    return [f(n) for n in [0 .. N]]


# Find b_i in o_L such that P = \sum b_i f_i:
def regular_basis_coeffs_for(P):
    n = P.degree(Y)
    fs = regular_basis(n)
    u = vwdwo(n)
    bs = [0] * (n + 1)
    for i in [0 .. n]:
        # Since f_i(u_i) = 1; f_j(u_i) = 0 for all j \geq i:
        bs[i] = P(u[i]) - sum(bs[j] * fs[j](u[i]) for j in [0 .. i-1])
    return vector(o_L, bs)

In [36]:
regular_basis_coeffs_for(regular_basis(10)[-1])

(0, O(Pi^14), O(Pi^14), O(Pi^15), O(Pi^14), O(Pi^14), O(Pi^15), O(Pi^14), O(Pi^14), O(Pi^16), 1 + O(Pi^14))

In [37]:
from time import process_time


# Decorator to record execution time
def time_this(func, name=None):
    if name is None:
        name = func.__name__
    
    
    def wrap(*args, **kwargs):
        _start = process_time()
        result = func(*args, **kwargs)
        _end   = process_time()
        
        print(f"{_end-_start : .2f} sec: {name}")
        return result

    return wrap


In [38]:
def sigma_coeffs_for_basis(M):
    m = M + 1
    n = (M+2) * (M+1) / 2
    B = Matrix(o_L, m, n)
    
    @time_this
    def compute_sigma():
        return sigmaMatrix(M + 1)
    sigma = compute_sigma()

    @time_this
    def fill_B(B):
        nb_col = 0
        
        for j in [0 .. M]:
            for i in [0 .. j]:
                bs = regular_basis_coeffs_for(sigma[i, j])
                # Only fill in non-zero columns:
                if any(b != 0 for b in bs):                
                    for k, b in enumerate(bs):
                        B[k, nb_col] = b  # Columns of B are the generators
                    nb_col += 1
        
        return B[:, 0:nb_col]
    B = fill_B(B)

    @time_this
    def get_SNF():
        return B.smith_form() 
    A, P, Q = get_SNF()   # A = P B Q
    
    C = Matrix(L, A[:, 0:M+1])  # A = [C|0]
    return ~P * A  # S_M = o_L . f (~P * A)    
    return ~C * P  # nth column is (\beta_{0,n}, ..., \beta_{M,n})^T
                   # where f_n = \sum_{k=0}^M \beta_{k,n} g_k
                   # and g_0, ..., g_k is a basis for the o_L module

In [39]:
sigma_coeffs_for_basis_iter(5)

 0.00 sec: SNF M=1
 0.00 sec: SNF M=2
 0.00 sec: SNF M=3
 0.01 sec: SNF M=4
 0.01 sec: SNF M=5


[ 1 + O(Pi^20)             0             0             0             0             0             0             0             0             0]
[            0  1 + O(Pi^20)             0             0             0             0             0             0             0             0]
[            0             0  2 + O(Pi^20)             0             0             0             0             0             0             0]
[            0             0             0  1 + O(Pi^20)             0             0             0             0             0             0]
[            0             0             0             0  1 + O(Pi^20)             0             0             0             0             0]
[            0             0             0             0             0 Pi + O(Pi^21)             0             0             0             0]

In [41]:
sigma_coeffs_for_basis_iter(20)

 0.00 sec: SNF M=1
 0.00 sec: SNF M=2
 0.00 sec: SNF M=3
 0.01 sec: SNF M=4
 0.01 sec: SNF M=5
 0.02 sec: SNF M=6
 0.02 sec: SNF M=7
 0.03 sec: SNF M=8
 0.04 sec: SNF M=9
 0.05 sec: SNF M=10
 0.06 sec: SNF M=11
 0.08 sec: SNF M=12
 0.10 sec: SNF M=13
 0.12 sec: SNF M=14
 0.15 sec: SNF M=15
 0.17 sec: SNF M=16
 0.21 sec: SNF M=17
 0.25 sec: SNF M=18
 0.29 sec: SNF M=19
 0.35 sec: SNF M=20


21 x 111 dense matrix over 3-adic Eisenstein Extension Field in Pi defined by x^2 - 3 (use the '.str()' method to see the entries)